In [9]:
print("ok")

ok


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [14]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "first_proj"

# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_API_KEY="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"
# LANGSMITH_PROJECT="first_proj"
# OPENAI_API_KEY="<your-openai-api-key>"

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [16]:
while True:
    ques = input("type your question. If you want to quit tha chat write quit")
    if ques != "quit":
        print(llm.invoke(ques).content)
    else:
        print("goodbye take care yourself")
        break

I do not have access to your personal information, including your name.
goodbye take care yourself


In [17]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [18]:
stores = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

In [19]:
config = {"configurable": {"session_id": "firstchat"}}

In [21]:
model_with_memory = RunnableWithMessageHistory(llm,get_session_history)
model_with_memory.invoke(("Hi! I'm Anil kumar"), config=config).content

"Hello Anil kumar, it's nice to meet you."

In [22]:
model_with_memory.invoke(("tell me what is my name?"), config = config).content

'Your name is Anil kumar.'

## RAG

In [23]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from  langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [25]:
loader = DirectoryLoader("C:/Users/CRRao AIMSCS/Desktop", glob="./*.text", loader_cls=TextLoader)
docs = loader.load()

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function = len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

In [ ]:
db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})